<a href="https://colab.research.google.com/github/kenyam1979/Test-colab/blob/main/Test_RAPIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [ ]:
!nvidia-smi

Fri Nov 25 03:00:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuSignal
  1. BlazingSQL
  1. xgboost
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 300 (delta 74), reused 99 (delta 55), pack-reused 171
Receiving objects: 100% (300/300), 87.58 KiB | 14.60 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Traceback (most recent call last):
  File "rapidsai-csp-utils/colab/env-check.py", line 1, in <module>
    import pynvml
ModuleNotFoundError: No module named 'pynvml'


In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ge

In [ ]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:20
🔁 Restarting kernel...


In [ ]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

Found existing installation: cffi 1.15.1
Uninstalling cffi-1.15.1:
  Successfully uninstalled cffi-1.15.1
Found existing installation: cryptography 37.0.4
Uninstalling cryptography-37.0.4:
  Successfully uninstalled cryptography-37.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.1/427.1 kB 33.8 MB/s eta 0:00:00
Installing RAPIDS Stable 21.12
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=11.2
    - dask-sql
    - gcsfs
    - llvmlite
    - openssl
    - python=3.7
    - rapids=21.12


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------

# cuDF Examples #

In [9]:
import cudf
import io, requests
import pandas as pd
from sklearn.datasets import make_spd_matrix

In [10]:
#1000×1000のランダムな対称正定値行列を作る
random_1000 = make_spd_matrix(1000, random_state=None)

In [11]:
#Pandasでデータフレームを作成
df_random = pd.DataFrame(random_1000)
del random_1000

In [12]:
%%time
df_random.describe()

CPU times: user 4.25 s, sys: 29.5 ms, total: 4.28 s
Wall time: 5.42 s


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.027139,0.011040,-0.016817,0.004005,-0.041975,0.042288,0.003399,-0.003748,0.016823,0.080928,...,0.007120,0.013359,0.037997,-0.009635,-0.006103,-0.036286,0.000657,-0.014715,0.024146,0.025312
std,0.847895,0.330504,0.565223,0.106439,1.374422,1.334090,0.086547,0.146098,0.516641,2.576352,...,0.204711,0.405627,1.196835,0.335199,0.222044,1.191842,0.018739,0.497870,0.751383,0.788403
min,-3.610973,-1.412607,-2.178173,-0.445986,-5.343935,-5.693519,-0.344901,-0.566289,-2.203948,-10.998016,...,-0.869270,-1.727254,-5.092138,-1.321932,-0.853560,-4.637760,-0.034042,-1.934267,-3.208559,-3.365795
25%,-0.537394,-0.204427,-0.391573,-0.063294,-0.934932,-0.840576,-0.054697,-0.102762,-0.326870,-1.619116,...,-0.120971,-0.251990,-0.751340,-0.229194,-0.154297,-0.813509,-0.006163,-0.339545,-0.470796,-0.494942
50%,0.051762,0.021534,-0.037671,0.006582,-0.084366,0.083931,0.004536,-0.008771,0.030658,0.160670,...,0.011852,0.025151,0.071368,-0.017575,-0.014359,-0.071280,0.000092,-0.032783,0.046993,0.046767
75%,0.584383,0.223990,0.355634,0.071589,0.868044,0.911930,0.056445,0.093387,0.354959,1.764622,...,0.142221,0.275120,0.823522,0.209510,0.140928,0.755099,0.006591,0.311690,0.516630,0.539778
max,3.301945,1.281094,2.414693,0.513888,5.864688,5.176546,0.530604,0.629418,2.000446,10.014772,...,0.801785,1.571500,4.661211,1.426153,0.947184,5.088272,0.505057,2.115407,2.914642,3.061850


In [13]:
%%time 
df_random.mean()

CPU times: user 13 ms, sys: 0 ns, total: 13 ms
Wall time: 23.5 ms


0      0.027139
1      0.011040
2     -0.016817
3      0.004005
4     -0.041975
         ...   
995   -0.036286
996    0.000657
997   -0.014715
998    0.024146
999    0.025312
Length: 1000, dtype: float64

In [14]:
#Pandasデータフレームをcudf形式にする(GPUに載せる)
cudf_random = cudf.from_pandas(df_random)
del df_random

In [15]:
%%time
cudf_random.describe()

CPU times: user 12.9 s, sys: 41.9 ms, total: 13 s
Wall time: 15.3 s


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.027139,0.011040,-0.016817,0.004005,-0.041975,0.042288,0.003399,-0.003748,0.016823,0.080928,...,0.007120,0.013359,0.037997,-0.009635,-0.006103,-0.036286,0.000657,-0.014715,0.024146,0.025312
std,0.847895,0.330504,0.565223,0.106439,1.374422,1.334090,0.086547,0.146098,0.516641,2.576352,...,0.204711,0.405627,1.196835,0.335199,0.222044,1.191842,0.018739,0.497870,0.751383,0.788403
min,-3.610973,-1.412607,-2.178173,-0.445986,-5.343935,-5.693519,-0.344901,-0.566289,-2.203948,-10.998016,...,-0.869270,-1.727254,-5.092138,-1.321932,-0.853560,-4.637760,-0.034042,-1.934267,-3.208559,-3.365795
25%,-0.537394,-0.204427,-0.391573,-0.063294,-0.934932,-0.840576,-0.054697,-0.102762,-0.326870,-1.619116,...,-0.120971,-0.251990,-0.751340,-0.229194,-0.154297,-0.813509,-0.006163,-0.339545,-0.470796,-0.494942
50%,0.051762,0.021534,-0.037671,0.006582,-0.084366,0.083931,0.004536,-0.008771,0.030658,0.160670,...,0.011852,0.025151,0.071368,-0.017575,-0.014359,-0.071280,0.000092,-0.032783,0.046993,0.046767
75%,0.584383,0.223990,0.355634,0.071589,0.868044,0.911930,0.056445,0.093387,0.354959,1.764622,...,0.142221,0.275120,0.823522,0.209510,0.140928,0.755099,0.006591,0.311690,0.516630,0.539778
max,3.301945,1.281094,2.414693,0.513888,5.864688,5.176546,0.530604,0.629418,2.000446,10.014772,...,0.801785,1.571500,4.661211,1.426153,0.947184,5.088272,0.505057,2.115407,2.914642,3.061850


In [16]:
%%time
cudf_random.mean()

CPU times: user 252 ms, sys: 872 µs, total: 253 ms
Wall time: 254 ms


0      0.027139
1      0.011040
2     -0.016817
3      0.004005
4     -0.041975
         ...   
995   -0.036286
996    0.000657
997   -0.014715
998    0.024146
999    0.025312
Length: 1000, dtype: float64